In [1]:
import pandas as pd
import numpy as np
import datetime
from geopy.distance import vincenty

In [45]:
# buslines = ['S53','S93','S79+']
# buslines = ['X1','X2','X3','X4','X5','X6','X7','X8','X9','X10','X10B','X11','X12','X14','X15','X17','X17A','X17J','X19','X31','X42']
# buslines = ['X10','X10B','X11']
# buslines = ['S93']
buslines = ['S53']
# buslines = ['X17','X17A','X17J','X19']

In [46]:
busdata = pd.read_csv('May 2015 Bustime.csv')#.query('LONGITUDE > -74.072279')

In [47]:
busline_data = pd.DataFrame() 
for busline in buslines:
    busline_data = busline_data.append(busdata.query('INFERRED_ROUTE==@busline & LATITUDE != 0'))
del busdata

In [48]:
busline_data.sort_values(by=['BUS','DAY','TIME'], inplace=True)
busline_data.reset_index(inplace=True,drop=True)

In [49]:
busline_data['time_stamp'] = busline_data.apply(lambda x : datetime.datetime.strptime(str(x['DAY'])+str(x['TIME']).zfill(6), "%Y%m%d%H%M%S"), axis=1)

In [7]:
def mark_intersect(df,plane_ax,plane,thresh):
    if plane_ax=='lat':
        bools = busline_data['LATITUDE']>plane
    elif plane_ax=='lon':
        bools = busline_data['LONGITUDE']>plane
    inds = pd.Series(np.nan,index=df.index)
    tds = df['time_stamp'].diff()
    for i in df.index[:-1]:
        if bools[i]==bools[i+1]:
            continue
        elif bools[i]==False and bools[i+1]==True and df.loc[i]['BUS']==df.loc[i+1]['BUS'] and tds[i].seconds<=70:
            inds[i] = 1
        else:
            continue
    return inds

def get_speeds(marked_df):
    speeds = pd.DataFrame(np.nan,columns=['weekday','hour','speed'],index=marked_df.index)
    for i in marked_df.index[:-1]:
        if marked_df.loc[i,'BUS']==marked_df.loc[i+1,'BUS'] and marked_df.loc[i,'in_mark']==1 and marked_df.loc[i+1,'out_mark']==1:
            elapsed = marked_df.loc[i+1,'time_stamp'] - marked_df.loc[i,'time_stamp']
            dist = vincenty((marked_df.loc[i,'LATITUDE'],marked_df.loc[i,'LONGITUDE']),(marked_df.loc[i+1,'LATITUDE'],marked_df.loc[i+1,'LONGITUDE'])).miles
            if dist>0:
                speeds.loc[i,'weekday'] = marked_df.loc[i,'time_stamp'].weekday()
                speeds.loc[i,'hour'] = marked_df.loc[i,'time_stamp'].hour
                speeds.loc[i,'speed'] = np.divide(dist,np.divide(elapsed.seconds,3600))
            else:
                continue
    return speeds

In [8]:
# local cross-bridge routes, measure from bridge onramp almost to the turn
in_axis = 'lon'
in_plane = -74.053914 # Verraz onramp
out_axis = 'lat'
out_plane = 40.621604 # 87th street
thresh = 70 # max seconds since previous time_stamp, otherwise ignored
busline_data['in_mark'] = np.nan
busline_data['out_mark'] = np.nan
busline_data['in_mark'] = mark_intersect(busline_data,in_axis,in_plane,thresh)
busline_data['out_mark'] = mark_intersect(busline_data,out_axis,out_plane,thresh)
marked = busline_data.query('in_mark == 1 or out_mark == 1 and in_mark + out_mark != 2').reset_index(drop=True)

In [15]:
speeds_87th = get_speeds(marked)
speeds_87th.query('weekday <= 5').groupby('hour').median()

,weekday,speed
hour,,
0,2,18.082909
1,2,22.886785
2,2,23.325376
3,2,23.034190
4,2,19.583744
5,2,18.194967
6,2,16.322846
7,2,12.404095
8,2,12.715654


In [16]:
speeds_87th.query('weekday <= 5').groupby('hour').median().to_csv('bridge_to_86th.csv')

In [22]:
# express cross-bridge routes, from onramp to 36th street latitude
in_axis = 'lon'
in_plane = -74.053914 # Verraz onramp
out_axis = 'lat'
out_plane = 40.655521 # 36th street
thresh = 70 # max seconds since previous time_stamp, otherwise ignored
busline_data['in_mark'] = np.nan
busline_data['out_mark'] = np.nan
busline_data['in_mark'] = mark_intersect(busline_data,in_axis,in_plane,thresh)
busline_data['out_mark'] = mark_intersect(busline_data,out_axis,out_plane,thresh)
marked = busline_data.query('in_mark == 1 or out_mark == 1 and in_mark + out_mark != 2').reset_index(drop=True)

In [23]:
speeds_36th = get_speeds(marked)
speeds_36th.query('weekday <= 5').groupby('hour').median()

,weekday,speed
hour,,
0,2,46.826268
1,2,48.804101
2,2,48.669155
3,2,47.958586
4,2,46.576464
5,2,39.065340
6,2,30.955801
7,2,28.421255
8,2,25.505612


In [24]:
speeds_36th.query('weekday <= 5').groupby('hour').median().to_csv('bridge_to_36th.csv')

In [25]:
# express cross-bridge routes, from onramp to 59th street latitude
in_axis = 'lon'
in_plane = -74.053914 # Verraz onramp
out_axis = 'lat'
out_plane = 40.641549 # 59th street
thresh = 70 # max seconds since previous time_stamp, otherwise ignored
busline_data['in_mark'] = np.nan
busline_data['out_mark'] = np.nan
busline_data['in_mark'] = mark_intersect(busline_data,in_axis,in_plane,thresh)
busline_data['out_mark'] = mark_intersect(busline_data,out_axis,out_plane,thresh)
marked = busline_data.query('in_mark == 1 or out_mark == 1 and in_mark + out_mark != 2').reset_index(drop=True)

In [26]:
speeds_59th = get_speeds(marked)
speeds_59th.query('weekday <= 5').groupby('hour').median()

,weekday,speed
hour,,
0,2,44.851080
1,2,46.572774
2,2,46.626990
3,2,46.399186
4,2,45.623485
5,2,38.246711
6,2,31.211886
7,2,29.143766
8,2,25.210752


In [27]:
speeds_59th.query('weekday <= 5').groupby('hour').median().to_csv('bridge_to_59th.csv')

## gannon/SI-expy speed from willowbrook to verraz onramp

In [50]:
in_axis = 'lon'
in_plane = -74.146774 # Gannon
out_axis = 'lon'
out_plane = -74.053914 # Verraz onramp
thresh = 70 # max seconds since previous time_stamp, otherwise ignored
busline_data['in_mark'] = np.nan
busline_data['out_mark'] = np.nan
busline_data['in_mark'] = mark_intersect(busline_data,in_axis,in_plane,thresh)
busline_data['out_mark'] = mark_intersect(busline_data,out_axis,out_plane,thresh)
marked = busline_data.query('in_mark == 1 or out_mark == 1 and in_mark + out_mark != 2').reset_index(drop=True)

In [51]:
speeds_gannon = get_speeds(marked)
speeds_gannon.query('weekday <= 5').groupby('hour').median()

,weekday,speed
hour,,
4,2.0,52.047723
5,2.0,52.123584
6,2.0,26.843954
7,2.0,15.422958
8,2.0,21.121385
9,2.0,29.195465
10,2.0,32.187510
11,2.0,39.906574
12,2.0,47.028658


In [52]:
speeds_gannon.query('weekday <= 5').groupby('hour').median().to_csv('gannon_to_bridge_siexpy.csv')

In [8]:
in_axis = 'lon'
in_plane = -74.164914
out_axis = 'lon'
out_plane = -74.053914
thresh = 70 # max seconds since previous time_stamp, otherwise ignored
busline_data['in_mark'] = np.nan
busline_data['out_mark'] = np.nan
busline_data['in_mark'] = mark_intersect(busline_data,in_axis,in_plane,thresh)
busline_data['out_mark'] = mark_intersect(busline_data,out_axis,out_plane,thresh)
marked = busline_data.query('in_mark == 1 or out_mark == 1 and in_mark + out_mark != 2').reset_index(drop=True)

In [9]:
speeds0 = get_speeds(marked)
speeds0.query('weekday <= 5').groupby('hour').median()

,weekday,speed
hour,,
0,3.0,1.044504
1,2.0,9.732078
4,2.0,20.328110
5,2.0,52.145552
6,2.0,12.817897
7,2.0,12.799734
8,2.0,18.216141
9,2.0,19.114217
10,2.0,17.417134


In [9]:
speeds1 = get_speeds(marked)
speeds1.query('weekday <= 5').groupby('hour').median()

,weekday,speed
hour,,
0,2,46.826268
1,2,48.804101
2,2,48.669155
3,2,47.958586
4,2,46.576464
5,2,39.065340
6,2,30.955801
7,2,28.421255
8,2,25.505612


In [32]:
"""in_axis = 'lon'
in_plane = -74.053914
out_axis = 'lat'
out_plane = 40.620162
thresh = 70
busline_data['in_mark'] = np.nan
busline_data['out_mark'] = np.nan
busline_data['in_mark'] = mark_intersect(busline_data,in_axis,in_plane,thresh)
busline_data['out_mark'] = mark_intersect(busline_data,out_axis,out_plane,thresh)
marked = busline_data.query('in_mark == 1 or out_mark == 1 and in_mark + out_mark != 2').reset_index(drop=True)"""

In [ ]:
# speeds_86th = get_speeds(marked)

In [10]:
in_axis = 'lat'
in_plane = 40.655521
out_axis = 'lat'
out_plane = 40.698739
thresh = 70
busline_data['in_mark'] = np.nan
busline_data['out_mark'] = np.nan
busline_data['in_mark'] = mark_intersect(busline_data,in_axis,in_plane,thresh)
busline_data['out_mark'] = mark_intersect(busline_data,out_axis,out_plane,thresh)
marked = busline_data.query('in_mark == 1 or out_mark == 1 and in_mark + out_mark != 2').reset_index(drop=True)

In [11]:
speeds2 = get_speeds(marked)
speeds2.query('weekday <= 5').groupby('hour').median()

,weekday,speed
hour,,
0,2,40.840751
1,2,44.806089
2,2,43.924665
3,2,40.149954
4,2,40.860448
5,2,36.910370
6,2,33.675233
7,2,29.378434
8,2,20.686724
